In [1]:
from pyspark.sql import SparkSession
from pyspark.ml import Pipeline
from pyspark.sql.functions import mean,col,split, col, regexp_extract, when, lit
from pyspark.ml.feature import StringIndexer
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.feature import QuantileDiscretizer



In [2]:
# File location and type
file_location = "/FileStore/tables/health.csv"
file_type = "csv"

# CSV options
infer_schema = "true"
first_row_is_header = "true"
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
health = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(file_location)
display(health)

fips,POP_ESTIMATE_2018,total_state_pop,"Active Physicians per 100,000 Population, 2018 (AAMC)","Total Active Patient Care Physicians per 100,000 Population, 2018 (AAMC)","Active Primary Care Physicians per 100,000 Population, 2018 (AAMC)","Active Patient Care Primary Care Physicians per 100,000 Population, 2018 (AAMC)","Active General Surgeons per 100,000 Population, 2018 (AAMC)","Active Patient Care General Surgeons per 100,000 Population, 2018 (AAMC)","Percentage of Active Physicians Who Are Female, 2018 (AAMC)","Percentage of Active Physicians Who Are International Medical Graduates (IMGs), 2018 (AAMC)","Percentage of Active Physicians Who Are Age 60 or Older, 2018 (AAMC)","MD and DO Student Enrollment per 100,000 Population, AY 2018-2019 (AAMC)","Student Enrollment at Public MD and DO Schools per 100,000 Population, AY 2018-2019 (AAMC)","Percentage Change in Student Enrollment at MD and DO Schools, 2008-2018 (AAMC)","Percentage of MD Students Matriculating In-State, AY 2018-2019 (AAMC)","Total Residents/Fellows in ACGME Programs per 100,000 Population as of December 31, 2018 (AAMC)","Total Residents/Fellows in Primary Care ACGME Programs per 100,000 Population as of Dec. 31, 2018 (AAMC)","Percentage of Residents in ACGME Programs Who Are IMGs as of December 31, 2018 (AAMC)","Ratio of Residents and Fellows (GME) to Medical Students (UME), AY 2017-2018 (AAMC)","Percent Change in Residents and Fellows in ACGME-Accredited Programs, 2008-2018 (AAMC)","Percentage of Physicians Retained in State from Undergraduate Medical Education (UME), 2018 (AAMC)",All Specialties (AAMC),Allergy & Immunology (AAMC),Anatomic/Clinical Pathology (AAMC),Anesthesiology (AAMC),Cardiovascular Disease (AAMC),Child & Adolescent Psychiatry** (AAMC),Critical Care Medicine (AAMC),Dermatology (AAMC),Emergency Medicine (AAMC),"Endocrinology, Diabetes & Metabolism (AAMC)",Family Medicine/General Practice (AAMC),Gastroenterology (AAMC),General Surgery (AAMC),Geriatric Medicine*** (AAMC),Hematology & Oncology (AAMC),Infectious Disease (AAMC),Internal Medicine (AAMC),Internal Medicine/Pediatrics (AAMC),Interventional Cardiology (AAMC),Neonatal-Perinatal Medicine (AAMC),Nephrology (AAMC),Neurological Surgery (AAMC),Neurology (AAMC),Neuroradiology (AAMC),Obstetrics & Gynecology (AAMC),Ophthalmology (AAMC),Orthopedic Surgery (AAMC),Otolaryngology (AAMC),Pain Medicine & Pain Management (AAMC),Pediatrics** (AAMC),Physical Medicine & Rehabilitation (AAMC),Plastic Surgery (AAMC),Preventive Medicine (AAMC),Psychiatry (AAMC),Pulmonary Disease (AAMC),Radiation Oncology (AAMC),Radiology & Diagnostic Radiology (AAMC),Rheumatology (AAMC),Sports Medicine (AAMC),Thoracic Surgery (AAMC),Urology (AAMC),Vascular & Interventional Radiology (AAMC),Vascular Surgery (AAMC),State/Local Government hospital beds per 1000 people (2019),Non-profit hospital beds per 1000 people (2019),For-profit hospital beds per 1000 people (2019),Total hospital beds per 1000 people (2019),Total nurse practitioners (2019),Total physician assistants (2019),Total Hospitals (2019),Internal Medicine Primary Care (2019),Family Medicine/General Practice Primary Care (2019),Pediatrics Primary Care (2019),Obstetrics & Gynecology Primary Care (2019),Geriatrics Primary Care (2019),Total Primary Care Physicians (2019),Psychiatry specialists (2019),Surgery specialists (2019),Anesthesiology specialists (2019),Emergency Medicine specialists (2019),Radiology specialists (2019),Cardiology specialists (2019),Oncology (Cancer) specialists (2019),"Endocrinology, Diabetes, and Metabolism specialists (2019)",All Other Specialties specialists (2019),Total Specialist Physicians (2019),ICU Beds,pop_fraction
1001,55601,9775742,217.1,196.7,77.2,71.2,7.6,6.9,28.50%,18.10%,34.40%,49.1,22.8,136.80%,82.10%,29.9,11.9,21.90%,0.6,18.40%,48.40%,120.7374364,0.500513209,1.615292629,5.426018199,3.321587661,0.989651118,1.410537226,1.512914928,4.561495383,0.796271015,17.95022373,2.070304638,4.231611677,0.477762609,1.785922133,1.001026418,15.5

In [3]:
health.printSchema()

root
-- fips: integer (nullable = true)
-- POP_ESTIMATE_2018: integer (nullable = true)
-- total_state_pop: integer (nullable = true)
-- Active Physicians per 100,000 Population, 2018 (AAMC): double (nullable = true)
-- Total Active Patient Care Physicians per 100,000 Population, 2018 (AAMC): double (nullable = true)
-- Active Primary Care Physicians per 100,000 Population, 2018 (AAMC): double (nullable = true)
-- Active Patient Care Primary Care Physicians per 100,000 Population, 2018 (AAMC): double (nullable = true)
-- Active General Surgeons per 100,000 Population, 2018 (AAMC): double (nullable = true)
-- Active Patient Care General Surgeons per 100,000 Population, 2018 (AAMC): double (nullable = true)
-- Percentage of Active Physicians Who Are Female, 2018 (AAMC): string (nullable = true)
-- Percentage of Active Physicians Who Are International Medical Graduates (IMGs), 2018 (AAMC): string (nullable = true)
-- Percentage of Active Physicians Who Are Age 60 or Older, 2018 (AAMC): string (nullable = true)
-- MD and DO Student Enrollment per 100,000 Population, AY 2018-2019 (AAMC): double (nullable = true)
-- Student Enrollment at Public MD and DO Schools per 100,000 Population, AY 2018-2019 (AAMC): double (nullable = true)
-- Percentage Change in Student Enrollment at MD and DO Schools, 2008-2018 (AAMC): string (nullable = true)
-- Percentage of MD Students Matriculating In-State, AY 2018-2019 (AAMC): string (nullable = true)
-- Total Residents/Fellows in ACGME Programs per 100,000 Population as of December 31, 2018 (AAMC): double (nullable = true)
-- Total Residents/Fellows in Primary Care ACGME Programs per 100,000 Population as of Dec. 31, 2018 (AAMC): double (nullable = true)
-- Percentage of Residents in ACGME Programs Who Are IMGs as of December 31, 2018 (AAMC): string (nullable = true)
-- Ratio of Residents and Fellows (GME) to Medical Students (UME), AY 2017-2018 (AAMC): double (nullable = true)
-- Percent Change in Residents and Fellows in ACGME-Accredited Programs, 2008-2018 (AAMC): string (nullable = true)
-- Percentage of Physicians Retained in State from Undergraduate Medical Education (UME), 2018 (AAMC): string (nullable = true)
-- All Specialties (AAMC): double (nullable = true)
-- Allergy & Immunology (AAMC): double (nullable = true)
-- Anatomic/Clinical Pathology (AAMC): double (nullable = true)
-- Anesthesiology (AAMC): double (nullable = true)
-- Cardiovascular Disease (AAMC): double (nullable = true)
-- Child & Adolescent Psychiatry** (AAMC): double (nullable = true)
-- Critical Care Medicine (AAMC): double (nullable = true)
-- Dermatology (AAMC): double (nullable = true)
-- Emergency Medicine (AAMC): double (nullable = true)
-- Endocrinology, Diabetes & Metabolism (AAMC): double (nullable = true)
-- Family Medicine/General Practice (AAMC): double (nullable = true)
-- Gastroenterology (AAMC): double (nullable = true)
-- General Surgery (AAMC): double (nullable = true)
-- Geriatric Medicine*** (AAMC): double (nullable = true)
-- Hematology & Oncology (AAMC): double (nullable = true)
-- Infectious Disease (AAMC): double (nullable = true)
-- Internal Medicine (AAMC): double (nullable = true)
-- Internal Medicine/Pediatrics (AAMC): double (nullable = true)
-- Interventional Cardiology (AAMC): double (nullable = true)
-- Neonatal-Perinatal Medicine (AAMC): double (nullable = true)
-- Nephrology (AAMC): double (nullable = true)
-- Neurological Surgery (AAMC): double (nullable = true)
-- Neurology (AAMC): double (nullable = true)
-- Neuroradiology (AAMC): double (nullable = true)
-- Obstetrics & Gynecology (AAMC): double (nullable = true)
-- Ophthalmology (AAMC): double (nullable = true)
-- Orthopedic Surgery (AAMC): double (nullable = true)
-- Otolaryngology (AAMC): double (nullable = true)
-- Pain Medicine & Pain Management (AAMC): double (nullable = true)
-- Pediatrics** (AAMC): double (nullable = true)
-- Physical Medicine & Rehabilitation (AAMC): double (nullable = true)
-- Plastic Surgery (AAMC): double (n

In [4]:
print(health.count())

3142

In [5]:
## statistics descriptions

display(health.describe())

summary,fips,POP_ESTIMATE_2018,total_state_pop,"Active Physicians per 100,000 Population, 2018 (AAMC)","Total Active Patient Care Physicians per 100,000 Population, 2018 (AAMC)","Active Primary Care Physicians per 100,000 Population, 2018 (AAMC)","Active Patient Care Primary Care Physicians per 100,000 Population, 2018 (AAMC)","Active General Surgeons per 100,000 Population, 2018 (AAMC)","Active Patient Care General Surgeons per 100,000 Population, 2018 (AAMC)","Percentage of Active Physicians Who Are Female, 2018 (AAMC)","Percentage of Active Physicians Who Are International Medical Graduates (IMGs), 2018 (AAMC)","Percentage of Active Physicians Who Are Age 60 or Older, 2018 (AAMC)","MD and DO Student Enrollment per 100,000 Population, AY 2018-2019 (AAMC)","Student Enrollment at Public MD and DO Schools per 100,000 Population, AY 2018-2019 (AAMC)","Percentage Change in Student Enrollment at MD and DO Schools, 2008-2018 (AAMC)","Percentage of MD Students Matriculating In-State, AY 2018-2019 (AAMC)","Total Residents/Fellows in ACGME Programs per 100,000 Population as of December 31, 2018 (AAMC)","Total Residents/Fellows in Primary Care ACGME Programs per 100,000 Population as of Dec. 31, 2018 (AAMC)","Percentage of Residents in ACGME Programs Who Are IMGs as of December 31, 2018 (AAMC)","Ratio of Residents and Fellows (GME) to Medical Students (UME), AY 2017-2018 (AAMC)","Percent Change in Residents and Fellows in ACGME-Accredited Programs, 2008-2018 (AAMC)","Percentage of Physicians Retained in State from Undergraduate Medical Education (UME), 2018 (AAMC)",All Specialties (AAMC),Allergy & Immunology (AAMC),Anatomic/Clinical Pathology (AAMC),Anesthesiology (AAMC),Cardiovascular Disease (AAMC),Child & Adolescent Psychiatry** (AAMC),Critical Care Medicine (AAMC),Dermatology (AAMC),Emergency Medicine (AAMC),"Endocrinology, Diabetes & Metabolism (AAMC)",Family Medicine/General Practice (AAMC),Gastroenterology (AAMC),General Surgery (AAMC),Geriatric Medicine*** (AAMC),Hematology & Oncology (AAMC),Infectious Disease (AAMC),Internal Medicine (AAMC),Internal Medicine/Pediatrics (AAMC),Interventional Cardiology (AAMC),Neonatal-Perinatal Medicine (AAMC),Nephrology (AAMC),Neurological Surgery (AAMC),Neurology (AAMC),Neuroradiology (AAMC),Obstetrics & Gynecology (AAMC),Ophthalmology (AAMC),Orthopedic Surgery (AAMC),Otolaryngology (AAMC),Pain Medicine & Pain Management (AAMC),Pediatrics** (AAMC),Physical Medicine & Rehabilitation (AAMC),Plastic Surgery (AAMC),Preventive Medicine (AAMC),Psychiatry (AAMC),Pulmonary Disease (AAMC),Radiation Oncology (AAMC),Radiology & Diagnostic Radiology (AAMC),Rheumatology (AAMC),Sports Medicine (AAMC),Thoracic Surgery (AAMC),Urology (AAMC),Vascular & Interventional Radiology (AAMC),Vascular Surgery (AAMC),State/Local Government hospital beds per 1000 people (2019),Non-profit hospital beds per 1000 people (2019),For-profit hospital beds per 1000 people (2019),Total hospital beds per 1000 people (2019),Total nurse practitioners (2019),Total physician assistants (2019),Total Hospitals (2019),Internal Medicine Primary Care (2019),Family Medicine/General Practice Primary Care (2019),Pediatrics Primary Care (2019),Obstetrics & Gynecology Primary Care (2019),Geriatrics Primary Care (2019),Total Primary Care Physicians (2019),Psychiatry specialists (2019),Surgery specialists (2019),Anesthesiology specialists (2019),Emergency Medicine specialists (2019),Radiology specialists (2019),Cardiology specialists (2019),Oncology (Cancer) specialists (2019),"Endocrinology, Diabetes, and Metabolism specialists (2019)",All Other Specialties specialists (2019),Total Specialist Physicians (2019),ICU Beds,pop_fraction
count,3142,3142,3142,3142,3142,3142,3142,3142,3142,3142,3142,3142,3031,2888,2973,2971,3142,3142,3142,3031,3142,2987,3142,2957,3142,3142,3142,3119,3119,3142,3142,3024,3142,3142,3142,2937,3142,3113,3142,3000,2989,3119,3119,3139,3142,2882,3142,3142,3142,3142,3010,3142,3119,3105,3066,3142,2971,3119,3142,3090,2798,3076,31

In [6]:
health.select("POP_ESTIMATE_2018").show()

+-----------------+
POP_ESTIMATE_2018|
+-----------------+
 55601|
 218022|
 24881|
 22400|
 57840|
 10138|
 19680|
 114277|
 33615|
 26032|
 44153|
 12841|
 23920|
 13275|
 14987|
 51909|
 54762|
 12277|
 10715|
 36986|
+-----------------+
only showing top 20 rows

In [7]:
health.select("states").show()

--------------------------------------------------------------------------- 
 Py4JJavaError Traceback (most recent call last)
 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 97 try : 
 ---> 98 return f ( * a , ** kw ) 
 99 except py4j . protocol . Py4JJavaError as e : 

 /databricks/spark/python/lib/py4j-0.10.9-src.zip/py4j/protocol.py in get_return_value (answer, gateway_client, target_id, name) 
 327 "An error occurred while calling {0}{1}{2}.\n" . 
 --> 328 format(target_id, ".", name), value)
 329 else : 

 Py4JJavaError : An error occurred while calling o388.select.
: org.apache.spark.sql.AnalysisException: cannot resolve '`states`' given input columns: [Active General Surgeons per 100,000 Population, 2018 (AAMC), Active Patient Care General Surgeons per 100,000 Population, 2018 (AAMC), Active Patient Care Primary Care Physicians per 100,000 Population, 2018 (AAMC), Active Physicians per 100,000 Population, 2018 (AAMC), Active Primary Care Physicians per 100,000 Population, 2018 (AAMC), All Other Specialties specialists (2019), All Specialties (AAMC), Allergy & Immunology (AAMC), Anatomic/Clinical Pathology (AAMC), Anesthesiology (AAMC), Anesthesiology specialists (2019), Cardiology specialists (2019), Cardiovascular Disease (AAMC), Child & Adolescent Psychiatry** (AAMC), Critical Care Medicine (AAMC), Dermatology (AAMC), Emergency Medicine (AAMC), Emergency Medicine specialists (2019), Endocrinology, Diabetes & Metabolism (AAMC), Endocrinology, Diabetes, and Metabolism specialists (2019), Family Medicine/General Practice (AAMC), Family Medicine/General Practice Primary Care (2019), For-profit hospital beds per 1000 people (2019), Gastroenterology (AAMC), General Surgery (AAMC), Geriatric Medicine*** (AAMC), Geriatrics Primary Care (2019), Hematology & Oncology (AAMC), ICU Beds, Infectious Disease (AAMC), Internal Medicine (AAMC), Internal Medicine Primary Care (2019), Internal Medicine/Pediatrics (AAMC), Interventional Cardiology (AAMC), MD and DO Student Enrollment per 100,000 Population, AY 2018-2019 (AAMC), Neonatal-Perinatal Medicine (AAMC), Nephrology (AAMC), Neurological Surgery (AAMC), Neurology (AAMC), Neuroradiology (AAMC), Non-profit hospital beds per 1000 people (2019), Obstetrics & Gynecology (AAMC), Obstetrics & Gynecology Primary Care (2019), Oncology (Cancer) specialists (2019), Ophthalmology (AAMC), Orthopedic Surgery (AAMC), Otolaryngology (AAMC), POP_ESTIMATE_2018, Pain Medicine & Pain Management (AAMC), Pediatrics Primary Care (2019), Pediatrics** (AAMC), Percent Change in Residents and Fellows in ACGME-Accredited Programs, 2008-2018 (AAMC), Percentage Change in Student Enrollment at MD and DO Schools, 2008-2018 (AAMC), Percentage of Active Physicians Who Are Age 60 or Older, 2018 (AAMC), Percentage of Active Physicians Who Are Female, 2018 (AAMC), Percentage of Active Physicians Who Are International Medical Graduates (IMGs), 2018 (AAMC), Percentage of MD Students Matriculating In-State, AY 2018-2019 (AAMC), Percentage of Physicians Retained in State from Undergraduate Medical Education (UME), 2018 (AAMC), Percentage of Residents in ACGME Programs Who Are IMGs as of December 31, 2018 (AAMC), Physical Medicine & Rehabilitation (AAMC), Plastic Surgery (AAMC), Preventive Medicine (AAMC), Psychiatry (AAMC), Psychiatry specialists (2019), Pulmonary Disease (AAMC), Radiation Oncology (AAMC), Radiology & Diagnostic Radiology (AAMC), Radiology specialists (2019), Ratio of Residents and Fellows (GME) to Medical Students (UME), AY 2017-2018 (AAMC), Rheumatology (AAMC), Sports Medicine (AAMC), State/Local Government hospital beds per 1000 people (2019), Student Enrollment at Public MD and DO Schools per 100,000 Population, AY 2018-2019 (AAMC), Surgery specialists (2019), Thoracic Surgery (AAMC), Total Active Patient Care Physicians per 100,000 Population, 2018 (AAMC), Total Hospitals (2019), Total Primary Care Physicians (2019), Total Residents/Fellows in ACGME Programs per 100,000 Population